# Simuleer een epidemie



<div class="alert alert-box alert-success">
In dit project zullen we bestuderen hoe ziektes zich kunnen verspreiden doorheen een (sociaal) netwerk. We zullen onderzoeken hoe de structuur van een netwerk een invloed kan hebben op hoe snel een ziekte doorgegeven wordt. Finaal zullen we ook verschillende strategieën bekijken om de verspreiding van een ziekte tegen te gaan.
</div>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp
from scipy.spatial import distance_matrix

## Een ziekte-uitbraak in een sociaal netwerk

Laat ons nu kijken hoe we het SIR-ziekteverspreidingsmodel kunnen vertalen naar de taal van netwerken. <br> 
Aan de hand van een algemeen netwerk zullen we een veel realistischer model opstellen. Geen continue benadering meer! Dit model sluit verrasssend beter aan bij de werkelijkheid, en bovendien is het ook veel eenvoudiger om te bevatten en te simuleren. We kunnen een exacte oplossing bekomen zonder dat we afgeleiden of andere geavanceerde wiskundige technieken nodig hebben!

### Ziektedynamiek op een netwerk

In plaats van het aantal $S$-, $I$- en $R$-individuen doorheen de tijd bij te houden zoals bij het standaard SIR-model, zullen we voor elke knoop in het netwerk zijn of haar toestand bijhouden. De tijd zal niet continu variëren maar zal nu in discrete stappen voorbij gaan: $t = 0, 1, 2, 3, \ldots$. De toestand van knoop nummer $i$ op tijdstip $t$ wordt beschreven door $N_i^t\in \{S, I, R\}$. Dit wil zeggen dat knoop $i$ op tijdstip $t$ de toestand $S$ (vatbaar), $I$ (geïnfecteerd) of $R$ (resistent) kan hebben. De verandering in toestand van de knopen beschrijven we aan de hand van enkele eenvoudige regels. Analoog aan het oorspronkelijke SIR-model dat twee parameters heeft, beta (het infectiepercentage) en gamma (het herstelpercentage), heeft ook het netwerk SIR-model twee parameters.


#### Vatbare en geïnfecteerde mensen

Laten we ons eerst beperken tot vatbare en geïnfecteerde individuen. We gaan ervan uit dat vatbare individuen geïnfecteerd kunnen worden, en geïnfecteerde individuen kunnen resistent worden. Er is dus geen mogelijke overgang van geïnfecteerd naar vatbaar en ook niet van vatbaar naar resistent. Beschouw volgende regels:

1. Indien een knoop op tijdstip $t$ in toestand $S$ zit, dan heeft elke **geïnfecteerde** buur een kans $p_\text{inf}$ om de ziekte door te geven. De knoop gaat naar toestand $I$ indien minstens één buur de ziekte doorgeeft.
2. Indien een knoop op tijdstip $t$ in toestand $I$ zit, dan gaat deze naar de toestand $R$ met een kans $p_\text{res}$.

Dus, stel dat een knoop in toestand $S$ zit, en ze heeft $k$ buren die in toestand $I$ zitten. De kans dat geen enkele buur de ziekte doorgeeft, is dan:

$$
(1-p_\text{inf})^k,
$$

dus de kans dat de ziekte wel doorgegeven wordt, en er dus een transitie van toestand $S$ naar $I$ plaatsvindt, is:

$$
1 - (1-p_\text{inf})^k\,.
$$

We maakten hier gebruik van de productregel en de complementregel uit de kansrekening. 



#### Voorbeeld
Beschouw de knoop in het blauw omlijnd in het onderstaand voorbeeld. Stel dat $p_\text{inf}=0.2$, wat is dan de kans dat één van de drie zieke buren de ziekte doorgeeft?

![](.images/ziekteverspr.png)
<center> Figuur 4.</center>

Dit berekenen we met de volgende code:

In [ ]:
p_inf = 0.2
k = 3

p_ziekte_doorgegeven = 1 - (1 - p_inf)**k

print("kans om de ziekte te krijgen is", p_ziekte_doorgegeven)

Het effectief doorgeven van de ziekte kunnen we simuleren met NumPy, waar `np.random.rand()` een willekeurig getal, uniform verdeeld tussen 0 en 1, genereert. <br>We doen dat met de code in de volgende code-cel. Voer voor de simulatie die cel enkele keren uit.

In [ ]:
p_ziekte_doorgegeven > np.random.rand()

Bij `True` wordt de ziekte effectief doorgegeven, bij `False` niet. Merk op dat er een toevalsfactor in de simulatie is ingebouwd. 

> **Oefening 8**: Stel dat $p_\text{inf}=1$ (iedereen die ziek is, geeft direct de ziekte door aan al zijn of haar buren in het netwerk). Initieel zijn enkel knopen 1 en 11 geïnfecteerd in het voorbeeldnetwerk uit Figuur 3.<br> 
-  Wie is allemaal geïnfecteerd in de volgende stap? 
-  En in de stap daarna?

### Implementatie
We kunnen het model eenvoudig implementeren in Python m.b.v. SciPy. <br>Eerst zullen we een simpel sociaal netwerk genereren om dit model te illustreren:
-  We genereren daarvoor een populatie van `n` personen. Om het visueel te houden worden deze voorgesteld als punten in het $x,y$-vlak.
- Nadien genereren we een verbindingsmatrix die weergeeft of er een verbinding is tussen de knopen. 

#### Eerst zullen we de knopen van het netwerk genereren. We genereren terzelfdertijd de afstand tussen de knopen.

In [ ]:
def genereer_populatie(n):
    """Genereren van punten en bepalen van hun onderlinge afstand."""
    # n punten genereren, uniform in het xy-vlak
    X = np.random.rand(n, 2)
    # alle paarsgewijze afstanden tussen n punten
    D = distance_matrix(X, X)
    return X, D

In [ ]:
# populatie van netwerk van 200 punten genereren
n = 200
X, D = genereer_populatie(n)

In [ ]:
print(X,D)

De afstanden tussen twee personen vormen samen de afstandsmatrix $D$.

In [ ]:
# X bestaat uit 200 koppels en D is 200x200-matrix
print(X.shape, D.shape)

#### Nu zullen we de verbindingsmatrix V genereren.

Om een simpel model voor de verbindingsmatrix V te bekomen, laat ons zeggen dat de kans dat $v_{ij}=1$, dus dat knopen $i$ en $j$ verbonden zijn, gegeven wordt door:

$$
p_{ij} = \exp(-\alpha \, d_{ij})\,.
$$

**Hier geldt dat de kans op een verbinding tussen knopen $i$ en $j$ afneemt naarmate de afstand tussen de twee knopen toeneemt.** <br>
$\alpha$ is een parameter ($\alpha \geq 0$) die dit verband regelt. Een grote waarde van $\alpha$ zorgt ervoor dat twee ver uiteen gelegen knopen een heel kleine kans hebben om in verbinding te staan. Voor een kleine waarde van $\alpha$ is dit wel nog mogelijk. Bovendien geldt dat hoe groter de afstand is tussen twee knopen, hoe kleiner de kans op een verbinding. 

In [ ]:
# illustratie van effect van waarde van alpha
plt.figure() 

xwaarden = np.linspace(0, 10, 100)
plt.plot(xwaarden, np.exp(-0.1 * xwaarden), label=r"$\alpha=0.1$")       # r in omschrijving label omwille van LaTeX-code
plt.plot(xwaarden, np.exp(-0.5 * xwaarden), label=r"$\alpha=0.5$")
plt.plot(xwaarden, np.exp(-1 * xwaarden), label=r"$\alpha=1$")
plt.plot(xwaarden, np.exp(-5 * xwaarden), label=r"$\alpha=5$")
plt.plot(xwaarden, np.exp(-10 * xwaarden), label=r"$\alpha=10$")
plt.xlabel(r"Afstand $d_{ij}$")                 
plt.ylabel(r"Kans op verbinding $v_{ij}$")
plt.legend(loc=0)

plt.show()

> **Oefening 9**: Denk goed na over de betekenis van $\alpha$. Wat als $\alpha=0$? Wat als $\alpha$ heel groot is?

In [ ]:
def sample_verbindingsmatrix(D, alpha=1.0):
    """Genereren van verbindingsmatrix afhankelijk van de afstandsmatrix en alpha."""
   
    # verbindingsmatrix heeft dezelfde dimensie als de afstandsmatrix, beide zijn vierkant
    n = D.shape[1]             # aantal kolommen in D is gelijk aan de populatiegrootte
    
    # matrix aanmaken met 0 en 1 om verbindingen voor te stellen
    # alle elementen op de diagonaal zijn nul, matrix is symmetrisch
    A = np.zeros((n, n))
    
    for i in range(n):
        for j in range(i+1, n):
                 # kans op een verbinding
                 p = np.exp(- alpha * D[i,j])
                 # met een kans van p, maak een verbinding tussen i en j
                 if p > np.random.rand():
                        A[i,j] = 1
                        A[j,i] = 1      # symmetrische matrix
    return A


In [ ]:
# verbindingsmatrix van netwerk genereren voor alpha = 10
V = sample_verbindingsmatrix(D, alpha=10)
print(V)        # elke matrix kan gebruikt worden als representatie van een figuur  
print(V.min(), V.max())

In [ ]:
# visualiseren dat V uit nullen en enen bestaat
plt.imshow(V, cmap="gray")   # elke matrix kan gebruikt worden als representatie voor afbeelding, 0 zwart, 1 wit 

#### Het netwerk voorstellen met een graaf. 

Hiervoor schrijven we een nieuwe functie in Python.<br> Geïnfecteerde personen zullen weergegeven worden in het rood, resistente in het groen en vatbare in het geel. We zullen dus een gekleurde graaf gebruiken. <br>
Als de toestand van de knopen nog niet is meegegeven, kleuren we ze blauw.

Bij de lijst punten (knopen) van het netwerk, hoort dus ook een lijst van toestanden, waarbij de eerste toestand overeenkomt met de eerste knoop, de tweede toestand met de tweede knoop, enz. 

In [ ]:
 def plot_netwerk(X, V, toestanden=None):
    """Graaf van het netwerk.""" 
    n = V.shape[1]          # populatiegrootte is gelijk aan aantal kolommen van V
    
    # van elke knoop kleur nagaan en lijst van maken
    if toestanden is None:
        # geen toestanden gegeven, alle noden zijn blauw
        knoop_kleuren = "blue"
    else:
        kleur_map = {"S" : "yellow", "I" : "red", "R" : "green"}    # dictionary
        knoop_kleuren = [kleur_map[toestand] for toestand in toestanden]
    
    
    plt.figure()
    
    plt.axis("off")  # bij graaf geen assen  
    
    # plot n knopen, eerste kolom van X bevat x-coördinaat, tweede kolom y-coördinaat in juiste kleur
    plt.scatter(X[:,0], X[:,1], color=knoop_kleuren, zorder=1)    # zorder=1: punten op bovenste layer van graaf
    
    # teken de verbindingen in het grijs
    # n is de populatiegrootte en V[i,j] is de waarde van de verbinding (0 of 1)
    # als V[i,j] = 1, dan lijnstuk tussen i-de en j-de knoop
    # dus van X i-de en j-de rij nodig, dus X[i,j] nodig met x'n in eerste kolom daarvan en y's in tweede
    for i in range(n):
        for j in range(i+1, n):
            if V[i,j] == 1:
                plt.plot(X[[i,j],0], X[[i,j],1], alpha=0.8, color="grey", zorder=0)    # zorder=0: lijnen onderste layer van graaf
    plt.scatter([], [], color="yellow", label="S")       # lege punten om labels al te kunnen tonen
    plt.scatter([], [], color="red", label="I")
    plt.scatter([], [], color="green", label="R")
    plt.legend(loc=0)
    
    plt.show()

In [ ]:
plot_netwerk(X, V)       # knopen en verbindingen van ons netwerk plotten, nog zonder toestanden

#### Laat ons nu aan elk van de knopen een initiële toestand toekennen.

Initieel is iedereen in toestand $S$, behalve vijf willekeurige personen die geïnfecteerd zijn.

In [ ]:
n_inf = 5  # initieel aantal geïnfecteerden

#lijst maken van initiële toestanden 
initiele_toestanden = ["S"] * n         # lijst maken van 200 S'n
initiele_toestanden[0:n_inf] = ["I"] * n_inf  # 5 S'n vervangen dootr I, maakt niet uit welke

In [ ]:
print(initiele_toestanden)
print(len(initiele_toestanden))

In [ ]:
plot_netwerk(X, V, initiele_toestanden)     # knopen en verbindingen van ons netwerk plotten, nu met initiële toestanden

#### Overgang van ene toestand naar andere

We hebben dus een functie nodig die telkens de toestand op tijdstip $t$ omzet naar de toestand op tijdstip $t+1$. Dit is een vrij ingewikkelde functie! De overgang tussen tijdstippen noemen we een *tijdstap*.

In [ ]:
def update_toestand(toestanden, V, p_inf=1, p_res=0):
    "Functie die toestand aanpast naar nieuwe toestand per tijdstap."
    n = len(toestanden)        # aantal toestanden is populatiegrootte
    nieuwe_toestanden = []     # maak lijst om de nieuwe toestanden in op te slaan
    
    for i, toestand in enumerate(toestanden):         # ga lijst toestanden af en houd overeenkomstige index bij
        if toestand == "S":                           # persoon i is vatbaar
            # tel aantal geïnfecteerden die persoon i kent
            n_inf_kennissen = 0
            for j in range(n):
                if V[i,j] == 1 and toestanden[j] == "I":     # als persoon i in contact met geïnfecteerde persoon
                    n_inf_kennissen += 1
            # kans dat persoon i ziek wordt door een zieke kennis
            p_ziekte = 1 - (1 - p_inf)**n_inf_kennissen
            # effectief besmet of niet
            if (p_ziekte > np.random.rand()):
                toestand = "I" 
            else:
                toestand = "S"
            nieuwe_toestanden.append(toestand)
        elif toestand == "I":                          # persoon i is vatbaar
            # persoon die geïnfecteerd is, kan resistent worden
            # effectief besmet of niet
            if (p_res > np.random.rand()):
                toestand = "R"  
            else:
                toestand = "I"
            nieuwe_toestanden.append(toestand)
        elif toestand == "R":                          # persoon i is resistent
            # resistente personen blijven resistent
            nieuwe_toestanden.append("R")
    
    return nieuwe_toestanden

In [ ]:
# initiële toestanden updaten voor bepaalde p_inf en p_res voor één tijdstap
p_inf = 0.1
p_res = 0.01

nieuwe_toestanden = update_toestand(initiele_toestanden, V, p_inf, p_res)

print("aantal infecties op t=0:", 5)
print("aantal infecties op t=1:", nieuwe_toestanden.count("I"))

In [ ]:
plot_netwerk(X, V, nieuwe_toestanden)         # knopen en verbindingen van ons netwerk plotten, nu met toestanden op t=1

#### Simulatie evolutie toestanden

We kunnen dit herhalen voor een hele reeks tijdstappen aan de hand van een for-lus:

In [ ]:
def simuleer_epidemie(init_toestanden, V, tijdstappen, p_inf=1, p_res=0):
    """Simulatie van evolutie toestanden."""
    # sla de toestanden op in een lijst van lijsten
    toestanden_lijst = [init_toestanden]     # lijst huidige toestanden wordt als eerste element in toestanden_lijst gestopt
    toestanden = init_toestanden
    for t in range(tijdstappen):
        toestanden = update_toestand(toestanden, V, p_inf, p_res)
        toestanden_lijst.append(toestanden)
    return toestanden_lijst

Laat ons dit eens doen voor 100 tijdstappen.

In [ ]:
# simulatie van evolutie toestanden van initiële toestand over 100 tijdstappen
simulatie = simuleer_epidemie(initiele_toestanden, V, 100, p_inf, p_res)   # nog steeds p_inf = 0.1 en p_res = 0.01

Laat ons nu eens naar enkele snapshots doorheen de tijd kijken (op tijdstappen 0, 10, 20, 50, 70 en 100).

In [ ]:
# verloop na 0, 10, 20, 50, 70 en 100 tijdstappen
for t in [0, 10, 20, 50, 70, 100]:
    toestanden = simulatie[t]             # simulatie is lijst van toestanden van toestanden
    print("tijdstip {}: {} geïnfecteerd, {} resistent".format(t, toestanden.count("I"), toestanden.count("R")))
    plot_netwerk(X, V, toestanden)
    

We kunnen de voortgang makkelijker opvolgen aan de hand van een grafiek. Laat ons eens kijken hoe de verhoudingen tussen vatbaren, geïnfecteerden en resistenten wijzigen doorheen de tijd:

In [ ]:
def plot_progressiekrommen(toestanden_lijst):
    """Evolutie cijfers."""
    tijdstappen = len(toestanden_lijst)     # aantal elementen in toestanden_lijst is gelijk aan aantal tijdstappen
    # tel het aantal personen voor elke toestand per tijdstap
    S = [toestanden.count("S") for toestanden in toestanden_lijst]
    I = [toestanden.count("I") for toestanden in toestanden_lijst]
    R = [toestanden.count("R") for toestanden in toestanden_lijst]
    
    plt.figure()
    
    plt.plot(range(tijdstappen), I, color="purple", label="I")
    plt.plot(range(tijdstappen), S, color="orange", label="S")
    plt.plot(range(tijdstappen), R, color="green", label="R")
    plt.legend(loc=0)
    plt.xlabel("Tijd")
    plt.ylabel("Aantal personen")
    
    plt.show()

In [ ]:
def plot_progressievlakken(toestanden_lijst):
    """Evolutie cijfers."""
    tijdstappen = len(toestanden_lijst)     # aantal elementen in toestanden_lijst is gelijk aan aantal tijdstappen
    # tel het aantal personen voor elke toestand per tijdstap
    S = [toestanden.count("S") for toestanden in toestanden_lijst]
    I = [toestanden.count("I") for toestanden in toestanden_lijst]
    R = [toestanden.count("R") for toestanden in toestanden_lijst]
    
    plt.figure()
    
    plt.stackplot(range(tijdstappen), I, S, R,
                    labels=["I", "S", "R"], colors=["red", "yellow", "lightgreen"])
    plt.legend(loc=0)
    plt.xlabel("Tijd")
    plt.ylabel("Aantal personen")
    
    plt.show()

In [ ]:
plot_progressiekrommen(simulatie)

In [ ]:
plot_progressievlakken(simulatie)

> **Oefening 10**: Indien er te snel te veel mensen ziek worden, kan het gezondheidsapparaat overrompeld worden, met catastrofale gevolgen! Om dit te vermijden wordt het principe van *social distancing* toegepast: mensen moeten sociaal contact zo veel mogelijk vermijden. Dit zorgt ervoor dat de ziekte trager wordt doorgegeven. 
- Je kan social distancing simuleren door $\alpha$ hoger te zetten, bv. op 25. Doe dit. Zie je waarom het resultaat *flatten-the-curve*-effect noemt?

<div class="alert alert-box alert-info">
    Wil je deze notebook downloaden, maar is het bestand te groot geworden door de grafieken?<br>
    Verwijder dan eerst de output van de cellen door in het menu <b>Cell > All output > Clear</b> te kiezen.
    Je kan de notebook ook opslaan als pdf of uitprinten, net zoals je met een webpagina zou doen.
</div>

<img src=".images/cclic.png" alt="Banner" align="left" style="width:100px;"/><br><br>
Deze notebook van M. Stock en F. wyffels voor Dwengo vzw is in licentie gegeven volgens een <a href="http://creativecommons.org/licenses/by-nc-sa/4.0/">Creative Commons Naamsvermelding-NietCommercieel-GelijkDelen 4.0 Internationaal-licentie</a>. 

![Dwengo vzw](.images/logodwengo.png)